In [1]:
%load_ext autoreload
import pandas as pd
import pickle
import os
from random import sample
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import os
%autoreload 2
import objaverse
import objaverse.xl as xl
import bpy

objaverse._VERSIONED_PATH=os.path.join('../.objaverse', "hf-objaverse-v1")
sns.set_style("darkgrid")
sns.set_context("notebook", font_scale=1.15)

# Filter only objects having one diffusion texture
##### ℹ️ This notebook requires `data/2-annotations_filtered_by_thumbnails.parquet`
This notebook downloads all the filtered objects and filter them based on the the textures they have.

In [2]:
annotations = pd.read_parquet('../data/2-annotations_filtered_by_thumbnails.parquet')

## Download the whole filtered dataset
This gonna take a while...

In [7]:
paths = objaverse.load_objects(
    annotations['uid'][:25000].to_list(), download_processes=256)

In [ ]:
def count_uv_maps(glb_object_name):
    obj = bpy.data.objects.get(glb_object_name)
    if not obj:
        raise ValueError(f"Object '{glb_object_name}' not found in the scene.")
    
    if obj.type != 'MESH':
        raise TypeError(f"Object '{glb_object_name}' is not a mesh.")

    mesh_data = obj.data
    uv_maps = mesh_data.uv_layers

    print(f"Object '{glb_object_name}' has {len(uv_maps)} UV map(s).")
    return len(uv_maps)

In [ ]:
def count_diffuse_textures(obj_name):
    obj = bpy.data.objects.get(obj_name)
    if not obj or obj.type != 'MESH':
        raise ValueError("Object not found or is not a mesh.")

    diffuse_texture_count = 0

    for slot in obj.material_slots:
        mat = slot.material
        if mat and mat.use_nodes:
            for node in mat.node_tree.nodes:
                if node.type == 'TEX_IMAGE':
                    # Look for connection to 'Base Color'
                    for output in node.outputs:
                        for link in output.links:
                            if link.to_socket.name == 'Base Color':
                                diffuse_texture_count += 1
    print(f"Object '{obj_name}' has {diffuse_texture_count} diffuse texture(s).")
    return diffuse_texture_count
